# Lab10: Word2Vec

link: https://nthu-datalab.github.io/ml/labs/10_Word2Vec/10_Word2Vec.html

112501533 黃思誠


## Import Library


In [1]:
import collections
import os
import urllib
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg as LA
import tensorflow as tf
from sklearn.manifold import TSNE
from tensorflow.python.keras import Input, Model
from tensorflow.python.keras.layers import InputLayer, Layer


2023-11-02 09:46:55.534130: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-02 09:46:55.534274: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-02 09:46:55.534308: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-02 09:46:55.543461: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Some constants
VOCABULARY_SIZE = 50000
DOWNLOAD_URL = "http://mattmahoney.net/dc/"
DATA_PATH = "./data"
FILE_NAME = "text8.zip"
EXPECTED_BYTES = 31344016


In [3]:
# Hyper-parameters
params = {
    "training_steps": 40000,
    "skip_step": 2000,
    "batch_size": 512,
    "embed_size": 512,
    "num_sampled": 256,
    "learning_rate": 1.0,
}


## Download & Get dataset dictionary


### Function for download and loading


In [4]:
# Download the dataset
def download(file_name, expected_bytes):
    """Download the dataset text8 if it's not already downloaded"""
    local_file_path = os.path.join(DATA_PATH, file_name)
    if os.path.exists(local_file_path):
        print("Dataset ready")
        return local_file_path
    file_name, _ = urllib.request.urlretrieve(
        os.path.join(DOWNLOAD_URL, file_name), local_file_path
    )
    file_stat = os.stat(local_file_path)
    if file_stat.st_size == expected_bytes:
        print("Successfully downloaded the file", file_name)
    else:
        raise Exception(
            "File "
            + file_name
            + " might be corrupted. You should try downloading it with a browser."
        )
    return local_file_path


# Read the data into a list of strings.
def read_data(file_path):
    """Read data into a list of tokens"""
    with zipfile.ZipFile(file_path) as f:
        # tf.compat.as_str() converts the input into string
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data


# Build the dictionary and replace rare words with UNK token.
def build_dataset(words, n_words):
    """Create two dictionaries and count of occuring words
    - word_to_id: map of words to their codes
    - id_to_word: maps codes to words (inverse word_to_id)
    - count: map of words to count of occurrences
    """
    # map unknown words to -1
    count = [["UNK", -1]]
    # count of occurences for words in vocabulary
    count.extend(collections.Counter(words).most_common(n_words - 1))
    word_to_id = dict()  # (word, id)
    # record word id
    for word, _ in count:
        word_to_id[word] = len(word_to_id)
    id_to_word = dict(zip(word_to_id.values(), word_to_id.keys()))  # (id, word)
    return word_to_id, id_to_word, count


### Function for generate dataset dictionary


In [5]:
def convert_words_to_id(words, dictionary, count):
    """Replace each word in the dataset with its index in the dictionary"""
    data_w2id = []
    unk_count = 0
    for word in words:
        # return 0 if word is not in dictionary
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data_w2id.append(index)
    count[0][1] = unk_count
    return data_w2id, count


# utility function
def generate_sample(center_words, context_window_size):
    """Form training pairs according to the skip-gram model."""
    for idx, center in enumerate(center_words):
        context = np.random.randint(1, context_window_size)
        # get a random target before the center word
        for target in center_words[max(0, idx - context) : idx]:
            yield center, target
        # get a random target after the center word
        for target in center_words[idx + 1 : idx + context + 1]:
            yield center, target


def batch_generator(data, skip_window, batch_size):
    """Group a numeric stream into batches and yield them as Numpy arrays."""
    single_gen = generate_sample(data, skip_window)
    while True:
        center_batch = np.zeros(batch_size, dtype=np.int32)
        target_batch = np.zeros([batch_size, 1], dtype=np.int32)
        for idx in range(batch_size):
            center_batch[idx], target_batch[idx] = next(single_gen)
        yield center_batch, target_batch


In [7]:
# Create data directory
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

# Get the dataset
vocabulary = read_data(download(FILE_NAME, EXPECTED_BYTES))
word_to_id, id_to_word, count = build_dataset(vocabulary, VOCABULARY_SIZE)
data_w2id, count = convert_words_to_id(vocabulary, word_to_id, count)
del vocabulary  # reduce memory.


Dataset ready


## Construct the model


### Define the layers


In [8]:
# embedding matrix - hidden layer
class embedding_lookup(Layer):
    def __init__(self, embed_size):
        super(embedding_lookup, self).__init__()
        embedding_init = tf.keras.initializers.GlorotUniform()
        self.embedding_matrix = self.add_weight(
            name="embedding_matrix",
            trainable=True,
            shape=[VOCABULARY_SIZE, embed_size],
            initializer=embedding_init,
        )

    def call(self, inputs):
        center_words = inputs
        embedding = tf.nn.embedding_lookup(
            self.embedding_matrix, center_words, name="embedding"
        )
        return embedding


# context matrix - prediction layer
class nce_loss(Layer):
    def __init__(self, embed_size, num_sampled):
        super(nce_loss, self).__init__()
        self.embed_size = embed_size
        self.num_sampled = num_sampled

        nce_w_init = tf.keras.initializers.TruncatedNormal(
            stddev=1.0 / (embed_size**0.5)
        )
        self.nce_weight = self.add_weight(
            name="nce_weight",
            trainable=True,
            shape=[VOCABULARY_SIZE, self.embed_size],
            initializer=nce_w_init,
        )
        self.nce_bias = self.add_weight(
            name="nce_bias",
            trainable=True,
            shape=[VOCABULARY_SIZE],
            initializer=tf.keras.initializers.Zeros,
        )

    def call(self, inputs):
        embedding, target_words = inputs[0], inputs[1]
        loss = tf.reduce_mean(
            tf.nn.nce_loss(
                weights=self.nce_weight,
                biases=self.nce_bias,
                labels=target_words,
                inputs=embedding,
                num_sampled=self.num_sampled,
                num_classes=VOCABULARY_SIZE,
            ),
            name="loss",
        )
        return loss


### Define the model


In [9]:
class Word2Vec(Model):
    def __init__(
        self,
        embed_size=512,
        num_sampled=256,
    ):
        """To-Do: Define model variables"""
        super(Word2Vec, self).__init__()
        self.embed_size = embed_size
        self.num_sampled = num_sampled

        # define model structure
        self.center_words = InputLayer(
            input_shape=(), name="center_words", dtype=tf.int32
        )
        self.target_words = InputLayer(
            input_shape=(1,), name="target_words", dtype=tf.int32
        )

        self.embedding = embedding_lookup(self.embed_size)
        self.loss = nce_loss(self.embed_size, self.num_sampled)

    def call(self, inputs):
        """To-Do: Define data flow and return loss"""
        center_words, target_words = inputs[0], inputs[1]
        center_words = self.center_words(inputs[0])
        target_words = self.target_words(inputs[1])

        embedding = self.embedding(center_words)
        loss = self.loss([embedding, target_words])

        return loss


### Consruct model, loss, optimizer


In [10]:
word2vec = Word2Vec(embed_size=params["embed_size"], num_sampled=params["num_sampled"])
train_loss = tf.keras.metrics.Mean(name="train_loss")
optimizer = tf.keras.optimizers.legacy.SGD(
    learning_rate=params["learning_rate"], momentum=0.1, nesterov=True
)


2023-11-02 09:47:33.185136: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-02 09:47:33.207098: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-02 09:47:33.207133: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-02 09:47:33.207853: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-02 09:47:33.207879: I tensorflow/compile

## Train the model


### Create tensorflow dataset


In [11]:
def gen():
    yield from batch_generator(data_w2id, 2, params["batch_size"])


dataset = tf.data.Dataset.from_generator(
    gen,
    (tf.int32, tf.int32),
    (tf.TensorShape([params["batch_size"]]), tf.TensorShape([params["batch_size"], 1])),
).repeat()


### Training


In [12]:
# Define training function
@tf.function
def train_step(center_words, target_words):
    with tf.GradientTape() as tape:
        loss = word2vec([center_words, target_words])

    gradients = tape.gradient(loss, word2vec.trainable_variables)
    optimizer.apply_gradients(zip(gradients, word2vec.trainable_variables))

    train_loss(loss)


# Training
x = []
y = []
for step, (center_words, target_words) in enumerate(dataset):
    if step == params["training_steps"]:
        break
    train_step(center_words, target_words)

    if ((step + 1) % params["skip_step"]) == 0:
        template = "Step {:0}, Loss: {:.2f}"
        x.append(step + 1)
        y.append(train_loss.result())
        print(template.format(step + 1, train_loss.result()))
        train_loss.reset_states()


2023-11-02 09:47:43.262869: W tensorflow/tsl/platform/default/env.cc:299] We are not able to find a directory for temporary files.
Verify the directory access and available space under: /tmp. You can also provide a directory for temporary files with the environment variable TMP or TMPDIR. Example under bash: `export TMP=/my_new_temp_directory;`
2023-11-02 09:47:43.262925: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2023-11-02 09:47:43.262953: W tensorflow/tsl/platform/default/env.cc:299] We are not able to find a directory for temporary files.
Verify the directory access and available space under: /tmp. You can also provide a directory for temporary files with the environment variable TMP or TMPDIR. Example under bash: `export TMP=/my_new_temp_directory;`
2023-11-02 09:47:43.262971: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.c

Step 2000, Loss: 172.81
Step 4000, Loss: 28.64
Step 6000, Loss: 12.90


KeyboardInterrupt: 

## Plot the loss curve


In [ ]:
%matplotlib inline

plt.xlabel("steps")
plt.ylabel("loss")
plt.title("Learning Curve")
plt.plot(x, y)
plt.show()


## Visualizing the learned embeddings


In [ ]:
# get embedding matrix from model weights. > word2vec.weights[0]
embedding_matrix = word2vec.weights[0]


def plot_with_labels(low_dim_embs, labels):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=(18, 18), dpi=150)  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        plt.annotate(
            label,
            xy=(x, y),
            xytext=(5, 2),
            textcoords="offset points",
            ha="right",
            va="bottom",
        )


tsne = TSNE(
    perplexity=30,
    n_components=2,
    init="pca",
    n_iter=5000,
    method="exact",
    learning_rate="auto",
)
plot_only = 400
final_embeddings = embedding_matrix
low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
labels = [id_to_word[i] for i in range(plot_only)]
plot_with_labels(low_dim_embs, labels)


## Find top-5 nearest neighbors of two words.

Words: "will", "western"


In [ ]:
# handy method for calculating the similarity between 2 word
def cos_sim(word1, word2):
    id1 = word_to_id[word1]
    id2 = word_to_id[word2]

    vec1 = embedding_matrix[id1].numpy()
    vec2 = embedding_matrix[id2].numpy()

    return np.dot(vec1, vec2) / (LA.norm(vec1) * LA.norm(vec2))


def top_k_nearest(word, k):
    vec = embedding_matrix[word_to_id[word]]

    # calaulate cosine similarity  of `vec` and all other vocabularies
    dot = np.dot(embedding_matrix.numpy(), vec)
    embedding_norm = LA.norm(embedding_matrix.numpy(), axis=-1)
    vec_norm = LA.norm(vec)
    norm_product = embedding_norm * vec_norm
    cos_sim = dot / norm_product

    # print out top k nearest words
    indices = np.argsort(cos_sim)[::-1][:k]
    print("---top {} nearest words of {}---".format(k, word))
    for idx in indices:
        print(id_to_word[idx])
    print("\n")


top_k_nearest("will", 5)
top_k_nearest("western", 5)
